# LSTM Time Series Forecasting with PyTorch

This Jupyter Notebook provides a full implementation of training an LSTM model using PyTorch on your pivot table dataset. The steps include data preparation, model definition, training, and evaluation.

---

## Table of Contents

1. [Data Preparation](#1)
   - Import Libraries
   - Load and Preprocess Data
2. [Define Dataset Class](#2)
3. [Split Data into Training and Validation Sets](#3)
4. [Create DataLoaders](#4)
5. [Define the LSTM Model](#5)
6. [Initialize Model, Loss Function, and Optimizer](#6)
7. [Training Loop](#7)
8. [Evaluate the Model](#8)
9. [Save the Model](#9)
10. [Conclusion](#10)

---


<a id="1"></a>
## 1. Data Preparation

### Import Libraries


In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error


### Load and Preprocess Data

Assuming your pivot table dataset in "../data/processed/2023_complete_pivot.parquet", with dates as the index and country-brand pairs as columns.


In [31]:
# Load the dataset
df_pivot = pd.read_parquet("../data/processed/2023_complete_pivot.parquet")

# Reset index to turn the date into a column
df_pivot = df_pivot.reset_index()

# Rename the 'index' column to 'date'
df_pivot = df_pivot.rename(columns={'index': 'date'})

# Melt the DataFrame to long format
df = df_pivot.melt(id_vars=['date'], var_name='id', value_name='value')

# Sort by 'id' and 'date'
df = df.sort_values(['id', 'date']).reset_index(drop=True)

# Convert 'date' to datetime if not already
df['date'] = pd.to_datetime(df['date'])

# Display the first few rows
df.head()


,date,id,value
0,2013-01-01,Aldovia-AIMST,0.000000
1,2013-01-02,Aldovia-AIMST,0.006284
2,2013-01-03,Aldovia-AIMST,0.123459
3,2013-01-04,Aldovia-AIMST,0.055607
4,2013-01-05,Aldovia-AIMST,0.000000


<a id="2"></a>
## 2. Define Dataset Class


In [32]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        # Convert to torch tensors
        sequence = torch.FloatTensor(self.sequences[idx]).unsqueeze(-1)
        target = torch.FloatTensor([self.targets[idx]])
        return sequence, target


<a id="3"></a>
## 3. Split Data into Training and Validation Sets


In [35]:
# Determine the cutoff date for the split (e.g., last 20% for validation)
# Convert cutoff_date to numpy.datetime64
cutoff_date = np.datetime64(df['date'].quantile(0.8))

# Define sequence length
sequence_length = 30  # Adjust based on your data

# Initialize lists for training and validation data
train_sequences = []
train_targets = []
val_sequences = []
val_targets = []

for name, group in df.groupby('id'):
    group = group.sort_values('date').reset_index(drop=True)
    values = group['value'].values
    dates = group['date'].values
    cutoff_index = np.searchsorted(dates, cutoff_date)
    
    # Training data
    for i in range(cutoff_index - sequence_length):
        seq = values[i:i+sequence_length]
        target = values[i+sequence_length]
        train_sequences.append(seq)
        train_targets.append(target)
    
    # Validation data
    for i in range(cutoff_index, len(values) - sequence_length):
        seq = values[i:i+sequence_length]
        target = values[i+sequence_length]
        val_sequences.append(seq)
        val_targets.append(target)

print(f'Total training samples: {len(train_sequences)}')
print(f'Total validation samples: {len(val_sequences)}')


Total training samples: 6975716
Total validation samples: 1685552


<a id="4"></a>
## 4. Create DataLoaders


In [36]:
batch_size = 64  # Adjust based on your hardware capabilities

# Create dataset instances
train_dataset = TimeSeriesDataset(train_sequences, train_targets)
val_dataset = TimeSeriesDataset(val_sequences, val_targets)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


<a id="5"></a>
## 5. Define the LSTM Model


In [ ]:
class LSTMForecastingModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super(LSTMForecastingModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Define output layer
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Get the output from the last time step
        out = self.fc(out[:, -1, :])
        return out


<a id="6"></a>
## 6. Initialize Model, Loss Function, and Optimizer


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LSTMForecastingModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


<a id="7"></a>
## 7. Training Loop


In [ ]:
num_epochs = 20  # Adjust based on your needs

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    for sequences, targets in train_loader:
        sequences = sequences.to(device)
        targets = targets.to(device)
        
        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs.squeeze(), targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
    
    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for sequences, targets in val_loader:
            sequences = sequences.to(device)
            targets = targets.to(device)
            outputs = model(sequences)
            loss = criterion(outputs.squeeze(), targets)
            val_losses.append(loss.item())
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {np.mean(train_losses):.4f}, '
          f'Val Loss: {np.mean(val_losses):.4f}')


<a id="8"></a>
## 8. Evaluate the Model

### Calculate Metrics


In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for sequences, targets in val_loader:
        sequences = sequences.to(device)
        outputs = model(sequences)
        predictions.extend(outputs.squeeze().cpu().numpy())
        actuals.extend(targets.cpu().numpy())

mae = mean_absolute_error(actuals, predictions)
rmse = mean_squared_error(actuals, predictions, squared=False)

print(f'Validation MAE: {mae:.4f}')
print(f'Validation RMSE: {rmse:.4f}')


### Visualize Predictions


In [ ]:
# Plot the first 100 predictions vs actuals
plt.figure(figsize=(12, 6))
plt.plot(actuals[:100], label='Actual')
plt.plot(predictions[:100], label='Predicted')
plt.legend()
plt.title('Actual vs Predicted Values')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.show()


<a id="9"></a>
## 9. Save the Model


In [ ]:
torch.save(model.state_dict(), 'lstm_forecasting_model.pth')


<a id="10"></a>
## 10. Conclusion

You've successfully trained an LSTM model on your pivot table dataset using PyTorch. You can now use this model to make predictions on future data or further refine it by tuning hyperparameters or incorporating additional features.

---

## Additional Notes

- **Adjust Hyperparameters**: Experiment with `sequence_length`, `hidden_size`, `num_layers`, `batch_size`, and `num_epochs` to optimize performance.
- **Scaling Data**: If your data has varying scales, consider scaling or normalizing it.
- **Early Stopping**: Implement early stopping to prevent overfitting if necessary.
- **Feature Engineering**: Even without exogenous variables, adding features like day of the week or month can help capture temporal patterns.

## Example: Making Future Predictions


In [ ]:
def predict_future(model, initial_sequence, prediction_length):
    model.eval()
    predictions = []
    sequence = initial_sequence.copy()
    
    for _ in range(prediction_length):
        seq_input = torch.FloatTensor(sequence[-sequence_length:]).unsqueeze(0).unsqueeze(-1).to(device)
        with torch.no_grad():
            pred = model(seq_input)
        pred_value = pred.item()
        predictions.append(pred_value)
        sequence = np.append(sequence, pred_value)
    return predictions

# Example usage:
# Get the last sequence from the validation data
latest_sequence = val_sequences[-1]
future_predictions = predict_future(model, latest_sequence, prediction_length=7)

print("Future Predictions:", future_predictions)
